# Pose2Sim workshop
EnvisionBOX Summer School, 2025, Amsterdam. Presented by David Pagnon

<bt><br><br>

`Pose2Sim` provides a workflow for 3D markerless kinematics (human or animal), as an alternative to traditional marker-based MoCap methods. 

**Pose2Sim** is free and open-source, requiring low-cost hardware but with research-grade accuracy and production-grade robustness. It gives maximum control over clearly explained parameters. Any combination of phones, webcams, or GoPros can be used with fully clothed subjects, so it is particularly adapted to the sports field, the doctor's office, or for outdoor 3D animation capture.

***Note:*** For real-time analysis with a single camera, please consider **[Sports2D](https://github.com/davidpagnon/Sports2D)** (note that the motion must lie in the sagittal or frontal plane). 


In [ ]:
from IPython.display import Video, display

display(Video('Pose2Sim_small.mp4', embed=True, width=1280, height=720))

# Table of Contents

# Table of Contents

1. [Install and import libraries](#install-and-import-libraries)
   1. [Install MiniConda](#install-miniconda)
   2. [Clone the repository](#clone-the-repository)
   3. [Install Pose2Sim](#install-pose2sim)

2. [Try Pose2Sim Demo](#try-pose2sim-demo)
    1. [Copy the Pose2Sim demo files to the current folder](#copy-the-pose2sim-demo-files-to-the-current-folder)
    2. [Run Pose2Sim demo](#run-pose2sim-demo)
    3. [Understand what is happening](#understand-what-is-happening)
    4. [Visualize the results](#visualize-the-results)

3. [Run on the Tragic Talkers dataset](#run-on-the-tragic-talkers-dataset)
   1. [Choose a subset](#choose-a-subset)
   2. [Convert the calibration files to the Pose2Sim format](#convert-the-calibration-files-to-the-pose2sim-format)
   3. [Run Pose2Sim with model with hands](#run-pose2sim-with-model-with-hands)
   4. [Optional: Try it on a scene with multiple persons](#optional-try-it-on-a-scene-with-multiple-persons)

4. [Run on custom data](#run-on-custom-data)
   1. [Get your command line ready](#get-your-command-line-ready)
   2. [Record and share videos from phones](#record-and-share-videos-from-phones)
   3. [Organize the videos](#organize-the-videos)
   4. [Edit the Configuration files](#edit-the-configuration-files)
   5. [Run Pose2Sim](#run-pose2sim)

---

## Install and import libraries



### Install MiniConda

Conda will be required to install the dependencies of Pose2Sim.

Type `miniconda`in Google, click on the first, link, then "Download" on the upper right corner.\
Click `skip registration` and download the `Miniconda installer` (NOT Distribution installer!) for your operating system.

Follow the instructions to install it.


### Clone the repository

If not already done, you will need to clone the repository of the Summer School. If you did it already, please pull the last changes anyway (last step).

Type `install git` on Google and go to the first link.\
Download and install git.

Open `anaconda prompt`.\
Type `cd Desktop` (or any other location where you would like the Summer school files to be stored).\
Clone the repository: `git clone https://github.com/sarkadava/envisionBOX_SummerschoolAmsterdam2025.git`

Make sure you have the latest version: `git pull`




### Install Pose2Sim

***N.B.*** *The following instructions are the same as those in the [Pose2Sim documentation](https://github.com/perfanalytics/pose2sim)*
<br>

1. Open `anaconda prompt` 
2. Create a new conda environement:

In [ ]:
conda create -n Pose2Sim python=3.10 -y 
conda activate Pose2Sim

3. Install OpenSim:

In [ ]:
conda install -c opensim-org opensim -y

4. Also install the [OpenSim GUI](https://simtk.org/projects/opensim)

5. Install Pose2Sim:

In [ ]:
pip install pose2sim

6. *Optional:* Install the libraries for GPU support:

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
pip uninstall onnxruntime
pip install onnxruntime-gpu

---

## Try Pose2Sim Demo

### Copy the Pose2Sim demo files to the current folder

In [ ]:
!conda activate Pose2Sim
from Pose2Sim import Pose2Sim
from pathlib import Path
import os

singleperson_demo_path = Path(Pose2Sim.__file__).parent.resolve() / 'Demo_SinglePerson'


!cp -r {singleperson_demo_path} {Path.cwd()}/
os.chdir(Path.cwd() / 'Demo_SinglePerson')

   See how the folder is organized:
   ```
   Trial
   ├── calibration \
   ├── videos \
   └── Config.toml
    ```

### Run Pose2Sim demo

In [ ]:
Pose2Sim.calibration()
Pose2Sim.poseEstimation()
# Pose2Sim.synchronization() # skipped for now, as it does not work in a Notebook
Pose2Sim.personAssociation()
Pose2Sim.triangulation()
Pose2Sim.filtering()
Pose2Sim.markerAugmentation()
Pose2Sim.kinematics()

# Or equivalently:
#Pose2Sim.runAll() 
# Or:
#Pose2Sim.runAll(do_calibration=True, do_poseEstimation=True, do_synchronization=True, do_personAssociation=True, do_triangulation=True, do_filtering=True, do_markerAugmentation=True, do_kinematics=True)

### Understand what is happening

This runs all stages of the pipeline on the Demo data with the default parameters. :
- **Calibration:** Determining the **intrinsic parameters** of the cameras (focal length, optical center, distortion, ...) and their **extrinsic parameters** (location and orientation).\ 
  Here, we just convert a calibration file. Proper calibration will be done this afternoon when we collect our own data
- **Pose estimation:** Detecting the points of interest in each video (joint centers, face, etc)
- **Synchronization:** Make sure that the same time frame corresponds to the same body position.\
  I did not manage to make the tool work in Jupyter, so we are just going to skip it for now. We will do it again from the command line when we collect our own data.
- **Person association:** Make sure that person 1 is associated with person 1 across all cameras
- **Triangulation:** Transform all the 2D positions on videos to a 3D position in meters
- **Filtering:** Results can be somewhat noisy, especially if we do not have many cameras, if they are not optimally placed, or if calibration is not perfect.
- **Marker augmentation:** Add markers to the skeleton to facilitate the inverse kinematics stage.
- **Kinematics:** Obtain a biomechanically consistent animated skeleton with adjusted and fixed limb lengths, and joint angle constraints.

Have a look at the logs to understand what is going on.

### Visualize the results

Open the OpenSim GUI to visualize the results:

- File -> Open Model: Open the scaled `.osim` model from Demo_SinglePerson/kinematics
- File -> Load Motion: Load the `.mot` motion from Demo_SinglePerson/kinematics
- File -> Preview experimental data: Check the markers from Demo_SinglePerson/pose-3d


Optional:
- You can also open the .trc and .mot file in Excel or any other spreadsheet software.
- You can also look at the positions of cameras and the overlay of the skeleton and markers on videos by using the [Pose2Sim Blender add-on](https://github.com/davidpagnon/Pose2Sim_Blender). 

---

## Run on the Tragic Talkers dataset


Go back to the original Day1_MotionTracking directory

In [ ]:
os.chdir(Path.cwd().parent) 

### Choose a subset of the dataset to run on


In [ ]:
scene_name = 'femalemonologue2_t3' # Among 'conversation1_t3', 'femalemonologue1_t2', 'femalemonologue2_t3', 'interactive1_t2', 'interactive4_t3', 'male_monologue2_t3'*
cameras_to_use = ['01', '11', '12', '22'] # Among ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']

In [ ]:
from Pose2Sim import Pose2Sim
import shutil

sample_folder = Path.cwd().parent / 'Sample'
scene_folder = sample_folder/scene_name
calibration_folder = sample_folder/'camera_calibration_data'

Pose2Sim_scene_folder = Path.cwd() / ('Demo_'+scene_name)
Pose2Sim_scene_folder.mkdir(parents=True, exist_ok=True)
(Pose2Sim_scene_folder / 'calibration').mkdir(parents=True, exist_ok=True)
(Pose2Sim_scene_folder / 'videos').mkdir(parents=True, exist_ok=True)
demo_config_path = Path(Pose2Sim.__file__).parent.resolve() / 'Demo_SinglePerson' / 'Config.toml'
shutil.copyfile(demo_config_path, Pose2Sim_scene_folder / 'Config.toml')


### Convert the calibration files to the Pose2Sim format

Create the Pose2Sim folder structure

In [ ]:
from Pose2Sim.calibration import toml_write
from Pose2Sim.common import rotate_cam, world_to_camera_persp
import json
import cv2
import numpy as np

json_calib_files = sorted([list(calibration_folder.glob(f'*{nb}.json'))[0] for nb in cameras_to_use])
ret, C, S, D, K, R, T = [], [], [], [], [], [], []
for file_path in json_calib_files:
    with open(file_path, 'r') as f:
        calib_data_cam = json.load(f)['camera']
        C.append(file_path.stem)  # Name
        S.append([int(calib_data_cam['width']), int(calib_data_cam['height'])]) # Size
        D.append([float(d) for d in calib_data_cam['distortion']][:4]) # Distortion
        K.append(np.array([[float(calib_data_cam['fx']), float(calib_data_cam['skew']), float(calib_data_cam['cx'])],
                   [0, float(calib_data_cam['fy']), float(calib_data_cam['cy'])],
                   [0, 0, 1]])) # Intrinsics
        rot_mat_cam = np.array([float(r) for r in calib_data_cam['r']]).reshape(3,3)
        t_cam = np.array([float(t) for t in calib_data_cam['t']])

        # Rotate cameras by Pi/2 around x in world frame
        # camera frame to world frame
        R_w, T_w = world_to_camera_persp(rot_mat_cam, t_cam)
        # x_rotate -Pi/2 and z_rotate Pi
        R_w_90, T_w_90 = rotate_cam(R_w, T_w, ang_x=-np.pi/2, ang_y=0, ang_z=np.pi)
        # world frame to camera frame
        R_c_90, T_c_90 = world_to_camera_persp(R_w_90, T_w_90)
        # store in R and T
        R.append(cv2.Rodrigues(R_c_90)[0].squeeze())
        T.append(t_cam)

toml_calib_file = Pose2Sim_scene_folder/'calibration'/f"Calib_{'_'.join(cameras_to_use)}.toml"
toml_write(toml_calib_file, C, S, D, K, R, T)


### Run Pose2Sim with model with hands

1. **Open the Config.toml model, search for the section `pose` -> `pose_model`, and set it to "whole_body".**\
Save and exit


2. **Run Pose2Sim.**

N.B.: Calibration does not need to be run as we just converted the calibration files.\
N.B.: Synchronization does not need to be run either as the cameras were already synchronized.

In [ ]:
os.chdir(Pose2Sim_scene_folder)
from Pose2Sim import Pose2Sim

# Pose2Sim.calibration()
Pose2Sim.poseEstimation()
# Pose2Sim.synchronization()
Pose2Sim.personAssociation()
Pose2Sim.triangulation()
Pose2Sim.filtering()
Pose2Sim.markerAugmentation()
Pose2Sim.kinematics()

3. **Check results in OpenSim GUI:**

- File -> Open Model: Open the scaled `.osim` model from Demo_SinglePerson/kinematics
- File -> Load Motion: Load the `.mot` motion from Demo_SinglePerson/kinematics
- File -> Preview experimental data: Check the markers from Demo_SinglePerson/pose-3d



4. Optional:
- You can also open the .trc and .mot file in Excel or any other spreadsheet software.
- You can also look at the positions of cameras and the overlay of the skeleton and markers on videos by using the [Pose2Sim Blender add-on](https://github.com/davidpagnon/Pose2Sim_Blender). 

### Optional: Try it on a scene with multiple persons

Reproduce the previous steps on another scene ("interactiv1_t2", for example)\
Make sure you set `project` -> `multi_person` to "true" in COnfig.toml.


**Question:**\
***What do you notice? Why did it not work well?***

**Answer:**\
We have plenty of cameras, but they are clustered in 2 almost identical locations. This makes it hard for the person association to work when they pass behind each other.\
Use the [Pose2Sim Blender add-on](https://github.com/davidpagnon/Pose2Sim_Blender) to visualize the problematic positions of cameras.

---

## Run on custom data

This time, we will run in the command line instead of from the notebook. As a reference, here is the documentation: [Pose2Sim documentation](https://github.com/perfanalytics/pose2sim).

### Get your command line ready

   It is easier to open PowerShell that Anaconda in a specific folder, so we are going to make sure conda is available in PowerShell.\
   Open `Anaconda Prompt` and type the following command:
   ```bash
   conda init powershell
   ```
   Now, you can go to any folder (e.g. `Day1_MotionTracking/Demo_customData`), right click on an empty space, and click `Open in Terminal`\
   `/!\` Make sure you don't forget to type `conda activate Pose2Sim` everytime.



### Record and share videos from phones:

4 people (hopefully) are going to volunteer for their phones to be used.\
Note that we could also use a GoPro, a webcam, or any camera, but sharing the files would be slightly more time consuming.

1. **Set the cameras up:**\
   Set them in a such a way that they can see the full scene with as few occlusions as possible or undesired people in the background.
2. **Record intrisic validation videos**:\
   Film the calibration checkerboard with each camera. Make sure that there is no switch of lens between when you are aiming at the center of the scene and when you are aiming at the checkerboard.\
   Also make sure that the checkerboard is oriented in as many directions as possible and covers a large part of the screen.\
3. **Record the scene:**\
   Measure something on the scene: squares on the floor, lines on the wall, ... It is also completely possible to temporarily put a table there, and then remove it.\
   Film it for about one second (the duration does not matter, we will extract a single frame).\
   /!\ IMPORTANT! Now, make sure the cameras are not moved anymore!
4. **Record a single person:**\
   It can be a person speaking in sign language, for example.\ 
   Synchronization would work better if the person is doing a fast vertical motion at some point.
5. **Record 2 people:**\
    Do any action you feel inspired to do.\
    Same instruction for synchronization.
6. **Share the videos:**\
   Each phone owner can share all the videos on the Whatsapp group (preferably in HD).\
   This would be intrinsic calibration, extrinsic calibration, single person, multiple persons videos.
7. Everyone from the group can then download the videos from Whatsapp 


### Organize the videos

In [ ]:
!conda activate Pose2Sim
! pip show Pose2Sim

Copy-paste the "Location" the previous command gave you.\
Open it in your file explorer.\
Search for "Pose2Sim", and copy-paste `Demo_SinglePerson` and `Demo_MultiPerson` to Day1_MotionTracking.

Now replace all the videos by the ones you just downloaded from Whatsapp and put them in the appropriate folders (`videos` and `calibration`).\ 
Rename them accordingly.



### Edit the Configuration files

Open both `Config.toml` files.

Edit in particular: 
- `project` -> `multi_person` (or not) depending on the videos you are going to run.
- `pose` -> `pose_model` to "whole_body" if you want finger motion.
- `calibration` -> `calibration_type` to "calculate"
- Change the `calibration` -> `calculate` -> `intrinsic` parameters according to the checkerboard you used.
- Change the `calibration` -> `calculate` -> `extrinsic` -> `scene` according to the dimensions of the scene you measured and filmed.


### Run Pose2Sim

Right click in an empty space in the Single (and lated, multi) person folder, and click `Open in Terminal`.

Type:
```
conda activate Pose2Sim
ipython
``` 

Run, one line at a time:

In [ ]:
from Pose2Sim import Pose2Sim
Pose2Sim.calibration()
Pose2Sim.poseEstimation()
Pose2Sim.synchronization()
Pose2Sim.personAssociation()
Pose2Sim.triangulation()
Pose2Sim.filtering()
Pose2Sim.markerAugmentation()
Pose2Sim.kinematics()

Follow the pipeline step by step:
- You'll see that calibration now tries to find the checkerboard corners, and then asks you to click (in the right order) on the points you measured in the scene.
- There is now also a synchronization GUI that asks you which body point is moving the most, and at the person you want to synchronize on.
- The other stages are unchanged

As before, you can visually check your results, either in OpenSim GUI, or in the [Pose2Sim Blender add-on](https://github.com/davidpagnon/Pose2Sim_Blender). 